In [ ]:
import os
import cv2
import numpy as np
import mahotas
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from joblib import dump

# Feature extractor using Haralick texture features
def extract_features(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found or unreadable: {image_path}")
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (128, 128))  # Resize for consistency
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick[:6]  # Use only top 6 features to reduce false wrinkle detection

# Load dataset
def load_dataset(dataset_path):
    X, y = [], []
    for label in os.listdir(dataset_path):
        label_folder = os.path.join(dataset_path, label)
        if not os.path.isdir(label_folder):
            continue
        for file in os.listdir(label_folder):
            img_path = os.path.join(label_folder, file)
            try:
                features = extract_features(img_path)
                X.append(features)
                y.append(label)
            except Exception as e:
                print(f"Skipping {img_path} due to error: {e}")
    return np.array(X), np.array(y)

# Main script
if __name__ == "__main__":
    dataset_path = # Change this path to your dataset

    print("[INFO] Loading dataset...")
    X, y = load_dataset(dataset_path)

    # Encode labels
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    # PCA to reduce dimensions
    pca = PCA(n_components=4)  # Reduce further to make classifier more robust
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Train SVM
    print("[INFO] Training model...")
    model = SVC(kernel='linear', probability=True)
    model.fit(X_train_pca, y_train)

    # Evaluate
    y_pred = model.predict(X_test_pca)
    print("\n[RESULT] Accuracy:", accuracy_score(y_test, y_pred))
    print("[RESULT] Classification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))

    # Save model and PCA
    dump(model, "wrinkle_model.joblib")
    dump(pca, "wrinkle_pca.joblib")
    dump(le, "wrinkle_label_encoder.joblib")
    print("[INFO] Model, PCA, and label encoder saved.")


[INFO] Loading dataset...
[INFO] Training model...

[RESULT] Accuracy: 0.9826086956521739
[RESULT] Classification Report:
              precision    recall  f1-score   support

  NoWrinkles       1.00      0.97      0.99        79
    Wrinkled       0.95      1.00      0.97        36

    accuracy                           0.98       115
   macro avg       0.97      0.99      0.98       115
weighted avg       0.98      0.98      0.98       115

[INFO] Model, PCA, and label encoder saved.


In [ ]:
import cv2
import numpy as np
import mahotas
from joblib import load

# Load saved model, PCA, and label encoder
model = load("wrinkle_model.joblib")
pca = load("wrinkle_pca.joblib")
le = load("wrinkle_label_encoder.joblib")

# Extract features from image
def extract_features(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found or unreadable: {image_path}")
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (128, 128))
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick[:6]

# Predict function
def predict_wrinkle(image_path):
    features = extract_features(image_path)
    features_pca = pca.transform([features])
    prediction = model.predict(features_pca)[0]
    prob = model.predict_proba(features_pca)[0]

    label = le.inverse_transform([prediction])[0]
    confidence = prob[prediction]

    # Custom logic: if label is 'wrinkled' and confidence > 0.98, keep it as 'wrinkled'
    if label.lower() == 'wrinkled' and confidence >= 0.99:
        label = 'wrinkled'
    elif label.lower() == 'wrinkled':
        # Optionally, you can downgrade it to 'maybe_wrinkled' or 'uncertain'
        label = 'not wrinkled'

    return label, round(confidence, 4)


# Example usage
if __name__ == "__main__":
    image_path = # Change to your test image path
    label, confidence = predict_wrinkle(image_path)
    print(f"[RESULT] Wrinkle Prediction: {label} (Confidence: {confidence})")


[RESULT] Wrinkle Prediction: not wrinkled (Confidence: 0.8022)
